# TSA Chapter 10: GARCH Multi-Step Forecast Convergence

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch10/TSA_ch10_garch_convergence/TSA_ch10_garch_convergence.ipynb)

This notebook demonstrates:
- Demonstrating how multi-step GARCH(1,1) volatility forecasts converge to the unconditional volatility.

In [ ]:
!pip install yfinance arch matplotlib numpy pandas -q

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from arch import arch_model
import os, warnings
warnings.filterwarnings('ignore')

In [ ]:
# Color scheme and style configuration
COLORS = {
    'blue': '#1A3A6E', 'red': '#DC3545', 'green': '#2E7D32',
    'orange': '#E67E22', 'gray': '#666666', 'purple': '#8E44AD',
}

plt.rcParams.update({
    'figure.facecolor': 'none', 'axes.facecolor': 'none',
    'savefig.facecolor': 'none', 'savefig.transparent': True,
    'axes.spines.top': False, 'axes.spines.right': False,
    'axes.grid': False, 'font.size': 10, 'axes.titlesize': 12,
    'axes.labelsize': 10, 'xtick.labelsize': 9, 'ytick.labelsize': 9,
    'legend.fontsize': 9, 'figure.dpi': 150, 'lines.linewidth': 1.2,
    'axes.linewidth': 0.6, 'legend.facecolor': 'none',
    'legend.framealpha': 0, 'legend.edgecolor': 'none',
})

def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    try:
        charts_path = os.path.join('..', '..', '..', 'charts', name)
        fig.savefig(f'{charts_path}.pdf', bbox_inches='tight', transparent=True, dpi=150)
        fig.savefig(f'{charts_path}.png', bbox_inches='tight', transparent=True, dpi=150)
    except Exception:
        pass
    print(f'Saved: {name}.pdf + .png')

def legend_outside(ax, ncol=3, y=-0.18):
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, y), ncol=ncol, frameon=False)

In [ ]:
# Download Bitcoin data
btc = yf.download('BTC-USD', start='2019-01-01', progress=False)
if isinstance(btc.columns, pd.MultiIndex):
    btc.columns = btc.columns.get_level_values(0)
btc['Return'] = btc['Close'].pct_change() * 100
btc.dropna(inplace=True)
print(f'BTC data: {len(btc)} daily observations')

# Fit GARCH(1,1) on full data
returns = btc['Return'].dropna()
am = arch_model(returns, vol='Garch', p=1, q=1, mean='Constant', dist='normal')
res = am.fit(disp='off')

omega = res.params['omega']
alpha = res.params['alpha[1]']
beta = res.params['beta[1]']
uncond_var = omega / (1 - alpha - beta)

# Multi-step forecasts from several starting points
horizons = np.arange(1, 61)
fig, ax = plt.subplots(figsize=(7, 3.5))

starts = [-200, -150, -100, -50]
colors_list = [COLORS['blue'], COLORS['red'], COLORS['green'], COLORS['orange']]
for idx, s in enumerate(starts):
    window = returns.iloc[:s]
    mod = arch_model(window, vol='Garch', p=1, q=1, mean='Constant', dist='normal')
    r = mod.fit(disp='off', update_freq=0)
    fcast = r.forecast(horizon=60)
    var_path = np.sqrt(fcast.variance.values[-1, :])
    label_date = window.index[-1].strftime('%Y-%m-%d')
    ax.plot(horizons, var_path, color=colors_list[idx], lw=1.0, label=f'Start: {label_date}')

ax.axhline(np.sqrt(uncond_var), color=COLORS['gray'], ls='--', lw=1.0,
           label='Unconditional $\\sigma$')
ax.set_xlabel('Forecast horizon (days)')
ax.set_ylabel('$\\hat{\\sigma}_{t+h}$')
ax.set_title('GARCH Multi-Step Forecasts Converge to Unconditional Volatility')
legend_outside(ax, ncol=3, y=-0.22)
fig.tight_layout()
save_chart(fig, 'garch_convergence')
plt.show()